In [97]:
#standard libs
import numpy as np
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random

#Sklearn
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF
from sklearn import decomposition, datasets, model_selection, preprocessing, metrics

Get random recipes for cold user

In [96]:
rand_df = pd.read_csv('INFO_FINAL.csv')
a = list(rand_df['recipe'].sample(n=5))
a #list of recipes to randomly rate

['Pork Chops with Crust of Onions',
 'Firehouse Chili',
 'Hobo Casserole',
 'Strawberry Heaven',
 'Thick Chocolate Pudding']

Load pickles!

In [98]:
#model

with open(r"nmf_130.pickle", "rb") as input_file:
    m = pickle.load(input_file)
Q = m.components_

In [99]:
#dictionary (for index and output)

with open(r"recipes_dict.pickle", "rb") as input_file:
    recipes_dict= pickle.load(input_file)
#recipes_dict

In [100]:
#R_vector for imp method

with open(r"R_vec.pickle", "rb") as input_file:
    R_vec= pickle.load(input_file)
#R_vec

Initialise cold user

In [7]:
recipes = a 

ratings = [5, 1 ,1 ,1, 1]

In [8]:
a

['Thick and Creamy New England Clam Chowder',
 'Wholesome Homemade Honey Whole Wheat Bread',
 'Smothered Pork Chops',
 'Creamy Coleslaw',
 'New York Crumb Cake']

Process cold start user with python (as done in app page, not in function -> output of this fed to func)

In [9]:
ratings = map(int, ratings) #needs to be int
user_rating = dict(zip(recipes,ratings)) 
user_rating

{'Thick and Creamy New England Clam Chowder': 5,
 'Wholesome Homemade Honey Whole Wheat Bread': 1,
 'Smothered Pork Chops': 1,
 'Creamy Coleslaw': 1,
 'New York Crumb Cake': 1}

In [10]:
visitor ='new_user'
user_input = pd.DataFrame(user_rating, index=[visitor])
user_input

,Thick and Creamy New England Clam Chowder,Wholesome Homemade Honey Whole Wheat Bread,Smothered Pork Chops,Creamy Coleslaw,New York Crumb Cake
new_user,5,1,1,1,1


Step by step testing of NMF function for in recommender page

In [11]:
recipe =pd.DataFrame(columns= recipes_dict.keys())
#recipe

In [12]:
#process recipe so dont get repeat cols with concat

recipe_cols = list(recipe.columns)
user_input_cols = list(user_input.columns)
user_

In [13]:
for col in recipe_cols:
        if col in  user_input_cols:
            recipe.drop([col], axis=1, inplace=True)
        else: 
            pass

In [14]:
new_user = pd.concat([recipe, user_input])
#new_user

In [15]:
#imputing user mean -> output is array to combine with R_vec

user1= new_user.T.fillna(new_user.mean(axis=1).round(2)).T.values
user1.max()

5.0

In [16]:
user_i = (user1 + R_vec)/2
user_i.max() #user_i max 5.0 !!!<3

5.0

In [17]:
R_vec.max()

5.0

In [18]:
user_P = m.transform(user_i)
user_R = np.dot( user_P, Q) #repickle model... 

Get the recommendations

In [19]:
reco = pd.DataFrame({'user_imputed':user_i[0], 'predicted_ratings':user_R[0]}, index = recipes_dict.keys())
reco #can see some basically the same, some quite different (from imputed..)

,user_imputed,predicted_ratings
1 Pan Fudge Cake,3.259375,3.259773
15 Bean Soup,3.400000,3.398051
15 Minute Garlic Lemon Chicken,3.200000,3.201630
15 Minute Shrimp Scampi,3.300000,3.301894
2 Ingredient Pumpkin Muffins,3.328571,3.331025
...,...,...
our favorite italian beef,5.000000,3.398086
roasted zucchini with thyme,2.750000,3.155045
strawberry filling,3.000000,3.397621
tastes like a slice of pizza dip,2.937500,3.336657


In [20]:
reco_10 = reco.sort_values(by ='predicted_ratings', ascending= False).iloc[:10,:]
recommendation =list(reco_10.index)
recommendation
#reco_10

#interestingly top ratings not super close to five..bc input was 2 and 1 perhaps?

['Whipped Chocolate Cream Icing',
 'Lemon Cake Pie',
 'Fluffy Morning Pancakes',
 'Creamy Chickpea Curry',
 'Tender Crock Pot Turkey Breast',
 'My Crock Pot Spaghetti Sauce',
 'Peanut Butter and Pickle Sandwich',
 'Bacon and Lettuce Stuffed Cherry Tomatoes',
 'Crock Pot Candy',
 'Cajun Chicken Alfredo']

In [24]:
#for loop to print out the key values! (below is the one which goes into func and returns output!)

#for k in recommendation:
    #if k in recipes_dict.keys():
        #print (recipes_dict[k])

In [22]:
recoreco = []

for k in recommendation:
    if k in recipes_dict.keys():
        recoreco.append(recipes_dict[k])

In [23]:
recoreco

['Whipped Chocolate Cream Icing~cing sugar"~"Sift icing sugar and cocoa into a bowl.", "Add whipping cream and stir to mix.", "With beater whip as for whipping cream.", "This amount will very generously cover a large Angel cake or any other large cake.")',
 'Lemon Cake Pie~"sugar", "butter", "all-purpose flour", "salt", "lemon peel", "fresh lemon juice", "eggs", "milk")~"Put rack on lowest position in oven.", "Preheat oven to 375.", "In a medium bowl, stir together melted butter and sugar.", "Stir in flour, salt, lemon peel and lemon juice.", "In a small bowl, beat egg yolks with milk until well blended, stir into lemon mixture.", "In another medium sized bowl, beat egg whites with an electric mixer until they hold distinct but moist peaks. Gently fold egg whites into lemon mixture. (Don\'t mix it too much.).", "Pour filling into pie crust.", "I always end up with more filling than will fit in the pie crust so I put it in a couple ceramic rammekins and cook it along with the pie.", \n"

Test random function for app!

In [25]:
#rand_rec= random.choice(list(recipes_dict.values()), size=k)

rand_rec = random.sample(list(recipes_dict.values()), 5)

In [26]:
rand_rec

['Chinese Chicken Lo Mein~"boneless skinless chicken breasts", "olive oil", "soy sauce", "eggs")~"In large, deep frying pan with lid, place chicken breasts seasoned with salt and pepper, into 1 Tbl. olive oil and 1 teaspoon sesame oil.", "Cook over medium heat, covered, for 20 minutes, turning ocaisionally, til chicken is done.", "Remove chicken from pan to cool slightly. Cut into bite-sized pieces, toss with 1 teaspoon sesame oil, cover and set aside. Do not drain frying pan. Add 1 1/2 Tbl. sesame oil to pan.", "In pot, boil 3 pkgs. Ramen noodles for 3 minutes.", "Drain and RINSE WELL.", \n"Drain again and add to frying pan with 2 Tbl. soy sauce, cole slaw mix, reserved chicken and 2 of the seasoning packets from noodles.", "Stir fry, stirring, over med-low heat for 3-5 minutes or til cabbage mix is tender and all is well mixed.", "Top each individual serving with sliced or chopped egg if desired.", "Also serve with additional soy sauce if desired.", "Makes 4-6 servings.", "*Do not su

Test top 100 dictionary!

In [27]:
with open(r"top100_dict.pickle", "rb") as input_file:
    top100_dict = pickle.load(input_file)

In [28]:
recipe_names = list(top100_dict.values())

In [29]:
recipe_names

['Bourbon Chicken~("boneless chicken breasts", "olive oil", "garlic clove", "ginger", "crushed red pepper flakes", "light brown sugar", "ketchup", "cider vinegar", "water", "soy sauce")~Editor\'s Note:  Named Bourbon Chicken because it was supposedly created by a Chinese cook who worked in a restaurant on Bourbon Street.", "Heat oil in a large skillet.", "Add chicken pieces and cook until lightly browned.", "Remove chicken.", "Add remaining ingredients, heating over medium Heat until well mixed and dissolved.", "Add chicken and bring to a hard boil.", "Reduce heat and simmer for 20 minutes.", "Serve over hot rice and ENJOY.")',
 'To Die for Crock Pot Roast~("beef roast", "water")~Place beef roast in crock pot.", "Mix the dried mixes together in a bowl and sprinkle over the roast.", "Pour the water around the roast.", "Cook on low for 7-9 hours.", "Optional tweaks:", "1. Use onion soup mix instead of ranch.", "2. Add one cup of red wine along with the water.", "3. Add potatoes, carrots,

Three NMF funcs

In [40]:
recipe = a 

ratings = [5, 1 ,1 ,1, 1]

In [41]:
ratingss = map(int, ratings) #needs to be int
user_rating = dict(zip(recipe,ratingss)) 

In [42]:
user_rating

{'Thick and Creamy New England Clam Chowder': 5,
 'Wholesome Homemade Honey Whole Wheat Bread': 1,
 'Smothered Pork Chops': 1,
 'Creamy Coleslaw': 1,
 'New York Crumb Cake': 1}

In [76]:
def NMF_new_user(user_rating, recipes_dict, R_vec):  #user rating zipped dict in app file
    
    """
    Creates a df with the user input.
    Takes the pickled dictionary index and the 
    user input to create a user df. Cols iterated
    through to remove duplicates before concat. 
    User df then imputed with user-wise and 
    recipe wise means, and averaged. 
    
    Future: impute with cat wise weighted mean
    """

    visitor ='new_user'
    user_input = pd.DataFrame(user_rating, index=[visitor])
    
    #create necessary df
    recipe =pd.DataFrame(columns=recipes_dict.keys()) #takes values from pickled dict and makes into col names

    #fix the issue of repeat cols
    recipe_cols = list(recipe.columns)
    user_input_cols = list(user_input.columns)
    #print(user_input_cols)
                           
    for col in recipe_cols:
        if col in  user_input_cols:
            recipe.drop([col], axis=1, inplace=True)
        else: 
            pass
    new_user = pd.concat([recipe, user_input]) 

     #impute user with values (steps below)
    user1 = new_user.T.fillna(new_user.mean(axis=1).round(2)).T.values
    user2 = R_vec #R_vec is one row of the col wise imputed df as an array
    user_i = (user1 + user2)/2
    user_i

    return(user_i) 

In [77]:
NMF_new_user(user_rating, recipes_dict, R_vec)

array([[3.259375, 3.4     , 3.2     , ..., 3.      , 2.9375  , 3.      ]])

In [90]:
def NMF_model(user_i,model, Q, recipes_dict): 

    '''
    takes imputer cold-start user vector 
    and transforms it using the model 
    to output a list of recommended recipe names

    Future: hybrid model and NLP (with ratings)
    '''

    user_P = model.transform(user_i)
    user_R = np.dot( user_P, Q)
    
    #df of predictions
    reco = pd.DataFrame({'user_imputed':user_i[0], 'predicted_ratings':user_R[0]}, index = recipes_dict.keys())
    reco_10 = reco.sort_values(by ='predicted_ratings', ascending= False).iloc[:10,:]
    recommendation =list(reco_10.index)

    #get values from dict based on recommendation (NOTE: probably needs work...)

    return (recommendation) #formatted as list

In [91]:
NMF_model(user_i, m, Q, recipes_dict)

['Whipped Chocolate Cream Icing',
 'Lemon Cake Pie',
 'Fluffy Morning Pancakes',
 'Creamy Chickpea Curry',
 'Tender Crock Pot Turkey Breast',
 'My Crock Pot Spaghetti Sauce',
 'Peanut Butter and Pickle Sandwich',
 'Bacon and Lettuce Stuffed Cherry Tomatoes',
 'Crock Pot Candy',
 'Cajun Chicken Alfredo']

In [94]:
def NMF_output(user_rating):

    '''
    Small iterative function to go through list 
    of recommendations and return list 
    of matched dict values (with recipe instructions)
    '''
    user_i = NMF_new_user(user_rating, recipes_dict, R_vec)
    recommendation = NMF_model(user_i,m, Q, recipes_dict)

    recoreco = []

    for k in recommendation:
        if k in recipes_dict.keys():
            recoreco.append(recipes_dict[k])

    return(recoreco)

In [95]:
NMF_output(user_rating)

['Whipped Chocolate Cream Icing~cing sugar"~"Sift icing sugar and cocoa into a bowl.", "Add whipping cream and stir to mix.", "With beater whip as for whipping cream.", "This amount will very generously cover a large Angel cake or any other large cake.")',
 'Lemon Cake Pie~"sugar", "butter", "all-purpose flour", "salt", "lemon peel", "fresh lemon juice", "eggs", "milk")~"Put rack on lowest position in oven.", "Preheat oven to 375.", "In a medium bowl, stir together melted butter and sugar.", "Stir in flour, salt, lemon peel and lemon juice.", "In a small bowl, beat egg yolks with milk until well blended, stir into lemon mixture.", "In another medium sized bowl, beat egg whites with an electric mixer until they hold distinct but moist peaks. Gently fold egg whites into lemon mixture. (Don\'t mix it too much.).", "Pour filling into pie crust.", "I always end up with more filling than will fit in the pie crust so I put it in a couple ceramic rammekins and cook it along with the pie.", \n"

In [ ]:
#note: in this file model called m, in the app files called m 